In [1]:
# import modules
import numpy as np
import pandas as pd
import os
from scipy.linalg import lstsq # To increase efficiency, and double check the results
import sympy as sp
import re
np.set_printoptions(precision=5,linewidth=100) # adjust setting for prettier printing
import tkinter as tk
import tkinter.filedialog
%gui tk
# create a gui to let user select the working folder
root=tk.Tk()
working_folder = tkinter.filedialog.askdirectory(parent=root,title='Please select a directory')
root.destroy() # close the gui after user selects the input 
print(working_folder)

In [2]:
working_folder='E:/Ender/coursework/photogrammetry/Ass2'

In [3]:
#extract files from the working folder
file_list=os.listdir(working_folder)
for i in file_list:
    if i.endswith('.icf'):
        icf=i # extract icf filename
    elif i.endswith('.prj'):
        prj=i # extract prj filename
    elif i.endswith('.ori'):
        ori=i # extract ori filename
    elif i.endswith('.cam'):
        cam=i # extract cam filename
    elif i.endswith('.gcp'):
        gcp=i

In [4]:
os.chdir(working_folder) # change the path to working_folder
# Read .prj file to get the max iteration times and threshold for stopping the iteration
with open(prj,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('>>|\ |\n',txt)) # split the text so that I can index the value.
max_iteration=np.int32(txt[-7]) # Max iteration times
maxSigma=np.float32(txt[-6]) # The maximum allowable difference between unit weight variances in successive iterations. If the difference is less than this value, then the iterations will stop.
print('max_iteration:', max_iteration)
print('maxSigma:', maxSigma)

# Read .icf to get xy (image coordinates)
xy_img=pd.read_csv(icf,sep='\t|\   |\  ',engine='python',names=['Image_ID','Point_ID','x','y','w1','w2','w3','w4'])
## Subset the xy for each image
xy_img_09=xy_img[xy_img['Image_ID']=='65_09_20180803.jpg'] # subset the dataframe to get xy for 65_09_20180803.jpg
xy_img_10=xy_img[xy_img['Image_ID']=='65_10_20180803.jpg'] # repeat the process for the other images.
xy_img_14=xy_img[xy_img['Image_ID']=='65_14_20180803.jpg']
xy_img_18=xy_img[xy_img['Image_ID']=='65_18_20180803.jpg']
# Read IOP based on the output from lab 1
with open(cam,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('\t|\n',txt)) # split the text so that I can index the value.
xp,yp,c=np.float32(txt[9:12]) # extract xp,yp,c

print(f'xp={xp}, yp={yp}, c={c}')

k1,k2,k3,p1,p2=np.float32(txt[22:27]) #extract distortion parameters
print(f'k1={k1}, k2={k2}, k3={k3}, p1={p1}, p2={p2}')
# Read GCP from .gcp
gcp=pd.read_csv(gcp,engine='python',sep='\t|\   ',names=['Point_ID','X','Y','Z'],usecols=[0,1,2,3])
X_gcp=np.array(gcp['X']) # extract x of gcp
Y_gcp=np.array(gcp['Y'])
Z_gcp=np.array(gcp['Z'])
# Read .ori as initial guess for EOP for image9,10,14,18
with open(ori,'r') as f:
    line=0
    for i in f:
        line=line+1 # count line number
        text=f.readline()
        if line==9: # image 9
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_09,phi_09,kpp_09,Xo_09,Yo_09,Zo_09=(text[1:7])#extract EOP (initial guess)
        if line == 10:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_10,phi_10,kpp_10,Xo_10,Yo_10,Zo_10=(text[1:7])#extract EOP (initial guess)
        if line == 14:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_14,phi_14,kpp_14,Xo_14,Yo_14,Zo_14=(text[1:7])#extract EOP (initial guess)
        if line ==18:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_18,phi_18,kpp_18,Xo_18,Yo_18,Zo_18=(text[1:7])#extract EOP (initial guess)

max_iteration: 200
maxSigma: 1e-12
xp=0.06722940504550934, yp=-0.11779332906007767, c=8.15868854522705
k1=-0.0002753081498667598, k2=8.035393875616137e-06, k3=-2.082014987081493e-07, p1=6.32564042462036e-05, p2=-7.169081800384447e-05


In [5]:
# angle conversion
def deg2rad(angle): return np.deg2rad(np.float32(angle)) # When I read the angles from the text files, they are string, so I need to convert the type first.
def rad2deg(angle): return np.rad2deg(angle) # convert radian to degree
def preprocessing(xy_img):
    '''
    This function is used to pre-process the data for LSA
    '''
    num_observation=len(xy_img['x']) #compute number of observation
    xx=np.array(xy_img['x']).astype(np.float32) # define x_img
    yy=np.array(xy_img['y']).astype(np.float32) # define y_img
    point_ID=np.array(xy_img['Point_ID']).astype(np.int16)
    # The following lines compute the distortion
    x_bar=xx-xp
    y_bar=yy-yp
    rd=np.sqrt(x_bar**2+y_bar**2) # distance from principle point
    dist_x=x_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p1*(rd**2+2*x_bar**2)+2*p2*x_bar*y_bar
    dist_y=y_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p2*(rd**2+2*y_bar**2)+2*p1*x_bar*y_bar
    # the following lines compute x_corrected and y_corrected
    x_corrected=xx-dist_x
    y_corrected=yy-dist_y
    return x_corrected,y_corrected, num_observation,point_ID
def get_matrix(x_corrected,y_corrected,xp,yp,num_observation,point_ID,X_gcp,Y_gcp):
    '''
    This function defines the f and A in f=AX for LSA
    '''
    x_temp=x_corrected-xp # define x in f
    y_temp=y_corrected-yp # define y in f
    f_ls=np.zeros((2*num_observation,1))
    A_ls=np.zeros((2*num_observation,8))
    for i in range(num_observation):
        n_gcp=point_ID[i]-1 # find the corresponding gcp ID
        # define f
        f_ls[2*i]=x_temp[i] 
        f_ls[2*i+1]=y_temp[i]
        # define A
        A_ls[2*i]=np.array([X_gcp[n_gcp],Y_gcp[n_gcp],1,0,0,0,-x_temp[i]*X_gcp[n_gcp],-x_temp[i]*Y_gcp[n_gcp]])
        A_ls[2*i+1]=np.array([0,0,0,X_gcp[n_gcp],Y_gcp[n_gcp],1,-y_temp[i]*X_gcp[n_gcp],-y_temp[i]*Y_gcp[n_gcp]])
    return f_ls,A_ls,x_temp,y_temp

In [6]:
# LSA: y=AX. y = [x,y].T X=[C1,C2,C3,C4...,C8].T
def LSA(xy_img_ID,X_gcp,Y_gcp,xp,yp):
    x_corrected,y_corrected, num_observation,point_ID=preprocessing(xy_img_ID) # call the preprocessing function in order to compute f and A in next line
    f_ls,A_ls,x_temp,y_temp=get_matrix(x_corrected,y_corrected,xp,yp,num_observation,point_ID,X_gcp,Y_gcp)# compute f and A and elements in f
    X_ls,sigma_new,rank,s=lstsq(A_ls,f_ls) # Compute X in f=AX. This function is more efficient
    # X_ls=np.linalg.inv(A_ls.T@A_ls)@A_ls.T@f # This line is equivalent to the previous line
    return X_ls,x_temp,y_temp,num_observation,point_ID


In [28]:
def get_EOP(X_gcp,Y_gcp,xy_img_ID):
    '''
    This function is the main function. It calls LSA and use the X from LSA to compute EOP
    '''
    #Call LSA function to get required parameters
    X_ls,x_temp,y_temp,num_observation,point_ID=LSA(xy_img_ID,X_gcp,Y_gcp,xp,yp) 
    C1,C2,C3,C4,C5,C6,C7,C8=X_ls.reshape(-1) # define elements in C matrix
    C_array=np.array([[C1,C2,C3],[C4,C5,C6],[C7,C8,1]]) # define C matrix
    if (-C1*C2+C4*C5)/(C7*C8)>=0:
        c_new=np.sqrt(-(C1*C2+C4*C5)/(C7*C8)) # compute new c
    else:
        c_new=c # use old c
        print('use input c')
    A=np.array([[1,0,0],[0,1,0],[0,0,-c_new]])@C_array # Compute A matrix (known)
    ATA=A.T@A #A.T@A=B.T@B B.T@B contains X0,Y0,Z0
    X0=-ATA[0][2]/ATA[0][0] # compute X0
    Y0=-ATA[1][2]/ATA[0][0] # compute Y0
    Z0=np.sqrt((ATA[2][2]/ATA[0][0])-X0**2-Y0**2) # compute Z0
    S=np.zeros((4,4))
    X_gcp_sub=[] # create a list for X_gcp in case the PointID is not 1-25
    Y_gcp_sub=[] # create a list for Y_gcp in case the PointID is not 1-25
    Z_gcp_sub=[] # create a list for Z_gcp in case the PointID is not 1-25
    for i in range(num_observation):
        n_gcp=point_ID[i]-1 # define the GCP ID
        # define xi array
        xi=np.array([x_temp[i],y_temp[i],-c_new]) 
        xi=xi/np.sqrt(xi@xi.T) # unit vector
        # define Xi array 
        Xi=np.array([X_gcp[n_gcp]-X0,Y_gcp[n_gcp]-Y0,0-Z0])
        Xi=Xi/np.sqrt(Xi@Xi.T)
        # the following lines compute S matrix. 
        S1_temp=np.array([[0,-xi[0],-xi[1],-xi[2]],
                         [xi[0],0,xi[2],-xi[1]],
                         [xi[1],-xi[2],0,xi[0]],
                         [xi[2],xi[1],-xi[0],0]]) # component 1 for computing S
        S2_temp=np.array([[0,-Xi[0],-Xi[1],-Xi[2]],
                         [Xi[0],0,-Xi[2],Xi[1]],
                         [Xi[1],Xi[2],0,-Xi[0]],
                         [Xi[2],-Xi[1],Xi[0],0]]) # component 2 for computing S
        # compute S
        S_temp=S1_temp.T@S2_temp 
        S=S+S_temp
        X_gcp_sub.append(X_gcp[n_gcp]) # add the x coordinate of gcp to X_gcp_sub
        Y_gcp_sub.append(Y_gcp[n_gcp]) # add the y coordinate of gcp to Y_gcp_sub
        Z_gcp_sub.append(Z_gcp[n_gcp]) # add the z coordinate of gcp to Z_gcp_sub
    # convert the data type to numpy array
    X_gcp_sub=np.array(X_gcp_sub) 
    Y_gcp_sub=np.array(Y_gcp_sub)
    Z_gcp_sub=np.array(Z_gcp_sub)
    # Compute eigen value and eigen vector of S
    eig_value,eig_vector=np.linalg.eig(S)
    index=np.where(eig_value==np.max(eig_value))
    qo,qx,qy,qz=eig_vector[:,index].reshape(-1) # define quaternion
    # Use quaternion to define rotation matrix R
    R=np.array([[qo**2+qx**2-qy**2-qz**2,2*qx*qy-2*qo*qz,2*qx*qz+2*qo*qy],
                [2*qx*qy+2*qo*qz, qy**2-qz**2+qo**2-qx**2, 2*qy*qz-2*qo*qx],
                [2*qx*qz-2*qo*qy,2*qy*qz+2*qo*qx, qz**2-qy**2-qx**2+qo**2]])
    # compute omega, phi, kappa from R
    omega=np.arctan2(-R[1,2],R[2,2])
    phi=np.arcsin(R[0,2])
    kappa=np.arctan2(-R[0,1],R[0,0])
    # convert radian to degree
    omega=rad2deg(omega) 
    phi=rad2deg(phi)
    kappa=rad2deg(kappa)
    # The following lines compute the Nx,Ny,D in ordert to compute residue
    Nx=R[0,0]*(X_gcp_sub-X0)+R[1,0]*(Y_gcp_sub-Y0)+R[2,0]*(Z_gcp_sub-Z0)
    Ny=R[0,1]*(X_gcp_sub-X0)+R[1,1]*(Y_gcp_sub-Y0)+R[2,1]*(Z_gcp_sub-Z0)
    D =R[0,2]*(X_gcp_sub-X0)+R[1,2]*(Y_gcp_sub-Y0)+R[2,2]*(Z_gcp_sub-Z0)
    # compute residue in x and y directions with collinearity eq
    res_x = x_temp + c_new*Nx/D
    res_y = y_temp + c_new*Ny/D
    # compare residue in x and y directions with projective transformation (linear form)
    res_x_c = x_temp - (C1*X_gcp_sub+C2*Y_gcp_sub+C3-x_temp*C7*X_gcp_sub-x_temp*C8*Y_gcp_sub)
    res_y_c = y_temp - (C4*X_gcp_sub+C5*Y_gcp_sub+C6-y_temp*C7*X_gcp_sub-y_temp*C8*Y_gcp_sub)
    '''
    This commented part is used to check if the computation of residues is correct
    print((C1*X_gcp_sub+C2*Y_gcp_sub+C3-x_temp*C7*X_gcp_sub-x_temp*C8*Y_gcp_sub))
    print('C',C1,C2,C3,C7,C8)
    print('GCP',X_gcp_sub[0],Y_gcp_sub[0])
    print('img:',x_temp[0],y_temp[0])
    print((C1*X_gcp_sub+C2*Y_gcp_sub+C3)/(C7*X_gcp_sub+C8*Y_gcp_sub+1))
    print((C7*X_gcp_sub+C8*Y_gcp_sub+1))
    '''
    # compare residue in x and y directions with projective transformation (non-linear form)
    res_x_c2= x_temp - (C1*X_gcp_sub+C2*Y_gcp_sub+C3)/(C7*X_gcp_sub+C8*Y_gcp_sub+1)
    res_y_c2= y_temp - (C4*X_gcp_sub+C5*Y_gcp_sub+C6)/(C7*X_gcp_sub+C8*Y_gcp_sub+1)
    # The following lines save residue to a dataframe. It will be saved as csv later
    df_res=dict()
    df_res['Point ID']=point_ID
    df_res['residue x']=res_x
    df_res['residue y']=res_y
    df_res['residue_c_x_linear']=res_x_c
    df_res['residue_c_y_linear']=res_y_c
    df_res['residue_c_x_nonlinear']=res_x_c2
    df_res['residue_c_y_nonlinear']=res_y_c2
    df_res=pd.DataFrame(df_res)
    # compute RMSE
    RMSE=np.sqrt(np.sum(res_x**2+res_y**2)/(num_observation-1))
    print('idx:',np.sum(res_x**2+res_y**2))
    RMSE_C=np.sqrt(np.sum(res_x_c**2+res_y_c**2)/(num_observation-1))
    print('linear',RMSE_C)
    RMSE_C2=np.sqrt(np.sum(res_x_c2**2+res_y_c2**2)/(num_observation-1))
    print('nonlinear',RMSE_C2)
    # Collect EOP into one list
    EOP=[X0,Y0,Z0,omega,phi,kappa]
    ### save output
    np.savetxt(f'{xy_img_ID}.eop',EOP,delimiter=',')
    np.savetxt(f'{xy_img_ID}_RMSE_c.csv',np.array([RMSE,RMSE_C,RMSE_C2,c_new]))
    df_res.to_csv(f'residue_{xy_img_ID}.res')
    return RMSE, EOP, df_res,c_new


In [27]:
RMSE_09, EOP_09,df_res_09,c_new_09=get_EOP(X_gcp,Y_gcp,xy_img_09)
print('RMSE:', RMSE_09)
print('c_new:', c_new_09)
print("EOP:", EOP_09)


[-5.57337 -3.01834 -0.58647  1.72026  3.92671 -5.15532 -2.8532  -0.64002  1.46929  3.50186 -4.81868
 -2.71469 -0.68562  1.25831  3.13956 -4.53506 -2.59742 -0.72281  1.0774   2.82658 -4.29478 -2.4961
 -0.75316  0.92345  2.55743]
C 1.8580000578263671 0.08942689366882606 -4.294782996894167 0.018039107724996672 -0.07624940042988654
GCP 0.0 3.778
img: -5.6112823 5.1974463
[-5.55803 -3.01946 -0.5919   1.71501  3.93516 -5.1546  -2.85319 -0.64311  1.4656   3.50285 -4.82077
 -2.71506 -0.68582  1.25687  3.13979 -4.53638 -2.597   -0.72244  1.07733  2.8266  -4.29478 -2.49642
 -0.75372  0.92357  2.55768]
[0.71193 0.72898 0.74606 0.76305 0.78015 0.78406 0.80111 0.81819 0.83518 0.85229 0.85581 0.87286
 0.88994 0.90694 0.92404 0.92817 0.94522 0.9623  0.9793  0.9964  1.      1.01705 1.03413 1.05112
 1.06822]
idx: 0.031174897408862555
0.02121085700344081
0.024467435099438845
RMSE: 0.03604100523990149
c_new: 7.279842397455152
EOP: [1.9777504969199495, 2.7749582369524513, 3.388498257987224, -16.9370631578

,Point ID,residue x,residue y,residue_c_x_linear,residue_c_y_linear,residue_c_x_nonlinear,residue_c_y_nonlinear
0,1,-0.093098,-0.016988,-0.037911,0.001172,-0.053251,0.001647
1,2,-0.015789,-0.015106,0.003013,0.003714,0.004133,0.005095
2,3,0.019559,-0.016938,0.015959,0.003499,0.021390,0.004690
3,4,0.036645,-0.031498,0.016908,-0.006547,0.022158,-0.008580
4,5,-0.009036,-0.047180,-0.029977,-0.018007,-0.038425,-0.023082
5,6,-0.026208,-0.043562,-0.002602,-0.019894,-0.003318,-0.025373
6,7,-0.011770,-0.022217,-0.000029,-0.002724,-0.000036,-0.003400
7,8,0.015470,-0.018247,0.013929,0.000937,0.017024,0.001145
8,9,0.030068,-0.007212,0.018706,0.010611,0.022397,0.012705
9,10,0.009417,0.004418,-0.005722,0.021153,-0.006714,0.024819


In [29]:
RMSE_10, EOP_10, df_res_10,c_new_10=get_EOP(X_gcp,Y_gcp,xy_img_10)
print('RMSE:', RMSE_10)
print('c_new:', c_new_10)
print("EOP:", EOP_10)
df_res_10


use input c
idx: 0.037596955952208244
linear 0.031420808128232135
nonlinear 0.030050248556728165
RMSE: 0.04043080828763708
c_new: 8.158689
EOP: [1.6599873736563886, 2.191996710002435, 3.514167926008604, -0.6753934906295416, -5.851494629585035, 92.06554295072304]


,Point ID,residue x,residue y,residue_c_x_linear,residue_c_y_linear,residue_c_x_nonlinear,residue_c_y_nonlinear
0,1,0.047611,0.052958,0.013293,0.018392,0.013518,0.018703
1,2,0.043862,0.022434,0.016744,-0.010287,0.016567,-0.010178
2,3,0.025770,0.005325,0.005627,-0.020590,0.005421,-0.019835
3,4,-0.001428,0.004313,-0.015747,-0.009130,-0.014782,-0.008571
4,5,-0.026600,0.052097,-0.035914,0.060853,-0.032870,0.055695
5,6,0.012722,0.034224,-0.011717,0.014961,-0.011865,0.015149
6,7,0.014941,0.019869,-0.000225,-0.000104,-0.000222,-0.000103
7,8,0.020429,-0.001472,0.015163,-0.017021,0.014549,-0.016332
8,9,0.014968,-0.018082,0.019408,-0.024388,0.018148,-0.022806
9,10,0.008691,0.002033,0.022817,0.012951,0.020804,0.011809


In [10]:
RMSE_14, EOP_14, df_res_14,c_new_14=get_EOP(X_gcp,Y_gcp,xy_img_14)
print('RMSE:', RMSE_14)
print('c_new:', c_new_14)
print("EOP:", EOP_14)
df_res_14

use input c
idx: 0.013468435244891892
0.02893008812216505
RMSE: 0.027354092975165417
c_new: 8.158689
EOP: [1.0090803317053887, 2.5091127158298696, 2.8064442206758944, -8.3439390075269, -17.747665309032538, 90.74214220325138]


,Point ID,residue x,residue y,residue_c_x_linear,residue_c_y_linear,residue_c_x_nonlinear,residue_c_y_nonlinear
0,3,0.042356,-0.001846,0.027405,-0.015736,0.027004,-0.015506
1,4,0.007496,0.003562,-0.002622,0.002653,-0.002339,0.002366
2,5,-0.008542,0.042674,-0.016870,0.063986,-0.013736,0.052098
3,7,0.010147,0.026257,0.001820,0.006721,0.001901,0.007018
4,8,0.013751,-0.006928,0.011135,-0.017366,0.010460,-0.016312
5,9,0.010348,-0.021352,0.013086,-0.025521,0.011175,-0.021795
6,10,0.011147,-0.003858,0.019792,0.004898,0.015487,0.003833
7,12,-0.037324,0.007250,-0.040966,-0.003491,-0.040673,-0.003466
8,13,-0.019911,-0.008119,-0.020519,-0.014797,-0.018418,-0.013281
9,14,-0.013095,-0.021243,-0.009322,-0.025032,-0.007638,-0.020511


In [11]:
RMSE_18, EOP_18, df_res_18,c_new_18=get_EOP(X_gcp,Y_gcp,xy_img_18)
print('RMSE:', RMSE_18)
print('c_new:', c_new_18)
print("EOP:", EOP_18)
df_res_18

use input c
idx: 0.044508963296393214
0.03525334485368549
RMSE: 0.04306437201852266
c_new: 8.158689
EOP: [2.0661646420713535, 1.2063402896991813, 3.713447206889112, 13.296219121301219, 0.41996876367626423, 90.07806183328822]


,Point ID,residue x,residue y,residue_c_x_linear,residue_c_y_linear,residue_c_x_nonlinear,residue_c_y_nonlinear
0,1,0.045756,0.047089,0.024015,0.016336,0.019177,0.013045
1,2,0.040876,0.024118,0.024347,-0.009496,0.019497,-0.007604
2,3,0.017208,0.008318,0.001325,-0.020625,0.001064,-0.016564
3,4,-0.015435,0.001817,-0.032715,-0.014548,-0.026349,-0.011718
4,5,-0.046835,0.035350,-0.064948,0.046886,-0.052462,0.037873
5,6,0.017456,0.032679,-0.002980,0.014020,-0.002506,0.011790
6,7,0.019486,0.022648,0.008557,0.002202,0.007218,0.001857
7,8,0.023297,0.008125,0.022204,-0.008955,0.018786,-0.007576
8,9,0.013910,-0.012985,0.020223,-0.021912,0.017162,-0.018595
9,10,0.002860,-0.001982,0.016414,0.009047,0.013972,0.007701
